# data parallelism
reference <https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html#sphx-glr-beginner-blitz-data-parallel-tutorial-py>

In [22]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [23]:
# parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("use device:", device)

use device: cpu


In [25]:
class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
    
    def __getItem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                        batch_size=batch_size, shuffle=True)

### Simple Model

In [26]:
class Model(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
    
    def forward(self, input):
        output = self.fc(input)
        print('\tIn Model: input size', input.size(),
              'output size', output.size())
        return output
    

In [27]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use ", torch.cuda.device_count(), "GPUs!")
model = torch.nn.DataParallel(model)

model.to(device)

DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [28]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

NotImplementedError: 